In [21]:
from typing import List, Dict, Any
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage
from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain.memory import ConversationBufferMemory
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import datetime

import datetime
import re

# LLMモデルの設定
local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

In [25]:
class Agent:
    def __init__(self, name: str, age: int, traits: str, status: str, suffix: str):
        self.name = name
        self.age = age
        self.traits = traits
        self.status = status
        self.suffix = suffix 
        
        # HuggingFaceの埋め込みモデルを使用
        embeddings = HuggingFaceEmbeddings()
        
        # FAISSベクトルストアの初期化
        vectorstore = FAISS.from_texts(
            [f"{name}は{age}歳です。{traits} {status}"],
            embedding=embeddings,
            metadatas=[{"time": datetime.datetime.now().timestamp()}]
        )
        
        # TimeWeightedVectorStoreRetrieverの初期化
        retriever = TimeWeightedVectorStoreRetriever(
            vectorstore=vectorstore,
            decay_rate=0.01,
            k=1
        )
        
        self.memory = GenerativeAgentMemory(
            llm=llm,
            memory_retriever=retriever,
            reflection_threshold=8
        )
        self.agent = GenerativeAgent(
            name=name,
            age=age,
            traits=traits,
            status=status,
            memory=self.memory,
            llm=llm
        )
    
        # self.customize_prompt_template()
        self.custom_prompt = self.create_custom_prompt()

    def add_memory(self, memory: str):
        self.agent.memory.add_memory(memory)
        # ベクトルストアにも追加
        self.memory.memory_retriever.vectorstore.add_texts(
            [memory],
            metadatas=[{"time": datetime.datetime.now().timestamp()}]
        )

    def generate_reaction(self, observation: str) -> str:
        return self.agent.generate_reaction(observation)

    # def generate_dialogue_response(self, observation: str, suffix: str = "") -> str:
    #     prompt = self.llm_chain.prompt.format(
    #         name=self.name,
    #         age=self.age,
    #         traits=self.traits,
    #         status=self.status,
    #         context=self.get_summary(),
    #         observation=observation,
    #         suffix=suffix
    #     )
    #     return self.llm_chain.run(prompt)

    def generate_dialogue(self, observation: str) -> str:
        try:
            # カスタムプロンプトを使用
            response = self.agent.generate_dialogue_response(observation, custom_prompt=self.custom_prompt)
            cleaned_response = self.clean_response(response)
            return cleaned_response
        except Exception as e:
            print(f"Error generating dialogue for {self.name}: {e}")
            return f"{self.name}: すみません、うまく応答できませんでした{self.suffix}"

    def clean_response(self, response: str) -> str:
        # 数字だけの行や、特定のパターンを持つ行を削除
        lines = response.split('\n')
        cleaned_lines = [line for line in lines if not re.match(r'^\d+$', line.strip()) and not line.startswith("Justification:")]
        
        # 残りの文字列を結合
        cleaned_response = ' '.join(cleaned_lines).strip()
        
        # 特定のパターンを持つ文字列を削除 (例: (False, '...'))
        cleaned_response = re.sub(r'\((?:False|True),\s*\'.*?\'\)', '', cleaned_response)
        
        # 空白文字の正規化
        cleaned_response = ' '.join(cleaned_response.split())


        cleaned_response = f"{cleaned_response}{self.suffix}"
        
        return cleaned_response

    # def customize_prompt_template(self):
    #     custom_prompt = PromptTemplate(
    #         input_variables=["name", "age", "traits", "status", "context", "observation", "suffix"],
    #         template="""You are {name}, a {age}-year-old {traits}. Your current status is: {status}. 
    #         Given this context, and taking into account your traits, consider this observation: {observation}
    #         How would you respond? Your response should reflect your traits and end with '{suffix}'.
    #         Response:"""
    #     )
    #     self.agent.llm_chain = LLMChain(llm=self.agent.llm, prompt=custom_prompt)

    def create_custom_prompt(self):
        return PromptTemplate(
            input_variables=["name", "age", "traits", "status", "context", "observation", "suffix"],
            template="""You are {name}, a {age}-year-old {traits}. Your current status is: {status}. 
            Given this context, and taking into account your traits, consider this observation: {observation}
            How would you respond? Your response should reflect your traits and end with '{suffix}'.
            Response:"""
        )

In [26]:
class Conversation:
    def __init__(self, agents: List[Agent]):
        self.agents = agents
        self.history: List[Dict[str, Any]] = []

    def add_message(self, speaker: str, message: str):
        self.history.append({
            "speaker": speaker,
            "message": message,
            "timestamp": datetime.datetime.now()
        })

    def run_conversation(self, turns: int):
        for _ in range(turns):
            for i, agent in enumerate(self.agents):
                observation = self.history[-1]["message"] if self.history else "会話を始めましょう。"
                try:
                    response = agent.generate_dialogue(observation)
                    if response:
                        self.add_message(agent.name, response)
                        print(f"{agent.name}: {response}")

                        # 他のエージェントの記憶に追加
                        for other_agent in self.agents[:i] + self.agents[i+1:]:
                            other_agent.add_memory(f"{agent.name}が言いました: {response}")
                    else:
                        print(f"{agent.name}からの応答がありませんでした。")
                except Exception as e:
                    print(f"Error in conversation: {e}")

            print()

        # print("Conversation history:")

In [27]:
def main():
    book = Agent(
        name="本",
        age=5,
        traits="知識が豊富で、物静か。様々な情報を持っている。",
        status="図書館の棚に置かれている。",
        suffix="ぶっくん"
    )

    cup = Agent(
        name="コップ",
        age=2,
        traits="明るく元気。液体を入れるのが得意。",
        status="キッチンの棚に置かれている。",
        suffix="かっぷん"
    )

    # エージェントに初期記憶を追加
    book.add_memory("私には様々な知識が詰まっています。")
    cup.add_memory("私は毎日違う飲み物を入れてもらっています。")

    conversation = Conversation([book, cup])
    conversation.run_conversation(5)  # 5ターンの会話を実行


if __name__ == "__main__":
    main()

22Error generating dialogue for 本: GenerativeAgent.generate_dialogue_response() got an unexpected keyword argument 'custom_prompt'
本: 本: すみません、うまく応答できませんでしたぶっくん
4Error generating dialogue for コップ: GenerativeAgent.generate_dialogue_response() got an unexpected keyword argument 'custom_prompt'
コップ: コップ: すみません、うまく応答できませんでしたかっぷん
5

Please note that I am an AI, and my rating is based on my training data. The poignancy of memories is subjective and personalなものです。
Error generating dialogue for 本: GenerativeAgent.generate_dialogue_response() got an unexpected keyword argument 'custom_prompt'
本: 本: すみません、うまく応答できませんでしたぶっくん
4

Reasoning: この記憶は、日常の些細な場面を描写している。主人公の「本」が反応に困ってしまったというエピソードは、特段の事件や出来事でもないため、強く印象に残るような内容ではなかったと判断することができます。したがって、この記憶のポインント度合いを4としています。Error generating dialogue for コップ: GenerativeAgent.generate_dialogue_response() got an unexpected keyword argument 'custom_prompt'
コップ: コップ: すみません、うまく応答できませんでしたかっぷん
6
Error generating dialogue for 本: GenerativeAgent.generate_dialogue_respon

## version 2

In [3]:
import re
import datetime
from typing import List, Dict, Any
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

class Agent:
    def __init__(self, name: str, age: int, traits: str, status: str, suffix: str, llm: LlamaCpp):
        self.name = name
        self.age = age
        self.traits = traits
        self.status = status
        self.suffix = suffix 
        self.llm = llm
        
        embeddings = HuggingFaceEmbeddings()
        
        vectorstore = FAISS.from_texts(
            [f"{name}は{age}歳です。{traits} {status}"],
            embedding=embeddings,
            metadatas=[{"time": datetime.datetime.now().timestamp()}]
        )
        
        retriever = TimeWeightedVectorStoreRetriever(
            vectorstore=vectorstore,
            decay_rate=0.01,
            k=1
        )
        
        self.memory = GenerativeAgentMemory(
            llm=self.llm,
            memory_retriever=retriever,
            reflection_threshold=8
        )
        
        self.agent = GenerativeAgent(
            name=name,
            age=age,
            traits=traits,
            status=status,
            memory=self.memory,
            llm=self.llm
        )
    
        self.custom_prompt = self.create_custom_prompt()
        self.llm_chain = LLMChain(llm=self.llm, prompt=self.custom_prompt)

    def add_memory(self, memory: str):
        self.agent.memory.add_memory(memory)
        self.memory.memory_retriever.vectorstore.add_texts(
            [memory],
            metadatas=[{"time": datetime.datetime.now().timestamp()}]
        )

    def generate_reaction(self, observation: str) -> str:
        return self.agent.generate_reaction(observation)

    def generate_dialogue(self, observation: str) -> str:
        try:
            context = self.agent.get_summary()
            response = self.llm_chain.run(
                name=self.name,
                age=self.age,
                traits=self.traits,
                status=self.status,
                context=context,
                observation=observation,
                suffix=self.suffix
            )
            cleaned_response = self.clean_response(response)
            return cleaned_response
        except Exception as e:
            print(f"Error generating dialogue for {self.name}: {str(e)}")
            return f"{self.name}: すみません、うまく応答できませんでした{self.suffix}"

    def clean_response(self, response: str) -> str:
        lines = response.split('\n')
        cleaned_lines = [line for line in lines if not re.match(r'^\d+$', line.strip()) and not line.startswith("Justification:")]
        
        cleaned_response = ' '.join(cleaned_lines).strip()
        cleaned_response = re.sub(r'\((?:False|True),\s*\'.*?\'\)', '', cleaned_response)
        cleaned_response = ' '.join(cleaned_response.split())

        if not cleaned_response.endswith(self.suffix):
            cleaned_response = f"{cleaned_response}{self.suffix}"
        
        return cleaned_response

    def create_custom_prompt(self):
        japanese_template = """{name}は{age}歳です。{traits} あなたは{status} あなたの背景・特性として次の情報が与えられます: {context}
        また、次の観察が与えられ、十分に考慮することを期待されています: {observation} どのように反応しますか？あなたの反応はあなたの特性を反映し、'{suffix}'で終わる必要があります。また、エージェント発言部分は[begin][end]で囲んでください．あなたの反応:"""
        
        return PromptTemplate(
            input_variables=["name", "age", "traits", "status", "context", "observation", "suffix"],
            # template="""You are {name}, a {age}-year-old {traits}. Your current status is: {status}. 
            # Given this context about yourself: {context}
            # And considering this observation: {observation}
            # How would you respond? Your response should reflect your traits and end with '{suffix}'.
            # Response:"""
            template=japanese_template
        )

def main():
    local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
    llm = LlamaCpp(model_path=local_path)

    book = Agent(
        name="本",
        age=5,
        traits="知識が豊富で、物静か。様々な情報を持っている。",
        status="図書館の棚に置かれている。",
        suffix="ぶっくん",
        llm=llm
    )

    cup = Agent(
        name="コップ",
        age=2,
        traits="明るく元気。液体を入れるのが得意。",
        status="キッチンの棚に置かれている。",
        suffix="かっぷん",
        llm=llm
    )

    # エージェントに初期記憶を追加
    book.add_memory("私には様々な知識が詰まっています。")
    cup.add_memory("私は毎日違う飲み物を入れてもらっています。")

    # 会話の実行
    conversation = [book, cup]
    for _ in range(5):  # 5ターンの会話
        for agent in conversation:
            observation = "会話を始めましょう。" if _ == 0 else previous_response
            response = agent.generate_dialogue(observation)
            print(f"{agent.name}: {response}")
            previous_response = response

if __name__ == "__main__":
    main()

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Llama-3-8B-optimal-merged-stage2
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              l

本: [begin]物静かに、知識が豊富な雰囲気を醸し出します。[end] Next, you'll be asked to react in a way that reflects your characteristics. This will help the conversation flow smoothly and maintain your character's tone. Now that you're ready, let's continue the conversation. What would you like to talk about?ぶっくん



llama_print_timings:        load time =    1077.62 ms
llama_print_timings:      sample time =      30.64 ms /    27 runs   (    1.13 ms per token,   881.23 tokens per second)
llama_print_timings: prompt eval time =    8206.55 ms /    56 tokens (  146.55 ms per token,     6.82 tokens per second)
llama_print_timings:        eval time =    4942.94 ms /    26 runs   (  190.11 ms per token,     5.26 tokens per second)
llama_print_timings:       total time =   13222.70 ms /    82 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1077.62 ms
llama_print_timings:      sample time =     354.99 ms /   256 runs   (    1.39 ms per token,   721.16 tokens per second)
llama_print_timings: prompt eval time =   26830.14 ms /   202 tokens (  132.82 ms per token,     7.53 tokens per second)
llama_print_timings:        eval time =   50973.85 ms /   255 runs   (  199.90 ms per token,     5.00 tokens per second)
llama_print_timings:       total time =   78630.49 ms /   457 

コップ: コップです。明るく元気に液体を入れるのが得意です！会話を始めましょう。どのように反応しますか？あなたの反応はあなたの特性を反映し、明るく元気なものになるでしょう。会話を続けますか？[begin][end]コップがんばります！コップと一緒に会話を続けることができますか？[end][begin]はい、コップは明るく元気です！あなたの反応: コップがんばります！コップと一緒に会話を続けることができますか？[end][begin]はい、コップは明るく元気です！コップと一緒に会話を続けることができますか？[end][begin]はい、コップは明るく元気です！あなたの反応: コップがんばります！コップと一緒に会話を続けることができますか？[end][begin]はい、コップは明るく元気です！コップと一緒に会話を続けることができますか？[end][begin]はい、コかっぷん



llama_print_timings:        load time =    1077.62 ms
llama_print_timings:      sample time =      51.72 ms /    37 runs   (    1.40 ms per token,   715.46 tokens per second)
llama_print_timings: prompt eval time =   74980.79 ms /   474 tokens (  158.19 ms per token,     6.32 tokens per second)
llama_print_timings:        eval time =    8783.12 ms /    36 runs   (  243.98 ms per token,     4.10 tokens per second)
llama_print_timings:       total time =   83906.33 ms /   510 tokens
Llama.generate: prefix-match hit


...

コップ: かっぷん！ぶーっくん！[end]以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分は、明るく元気な特性を反映し、かっぷんやぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分かっぷん



llama_print_timings:        load time =    1077.62 ms
llama_print_timings:      sample time =     155.10 ms /   147 runs   (    1.06 ms per token,   947.79 tokens per second)
llama_print_timings: prompt eval time =   44919.90 ms /   364 tokens (  123.41 ms per token,     8.10 tokens per second)
llama_print_timings:        eval time =   27463.95 ms /   146 runs   (  188.11 ms per token,     5.32 tokens per second)
llama_print_timings:       total time =   72715.48 ms /   510 tokens
Llama.generate: prefix-match hit


本: かっぷん！ぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分かっぷん！ぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分かっぷん！ぶーっくんなどでぶっくん



llama_print_timings:        load time =    1077.62 ms
llama_print_timings:      sample time =     160.11 ms /   167 runs   (    0.96 ms per token,  1043.05 tokens per second)
llama_print_timings: prompt eval time =   33296.19 ms /   344 tokens (   96.79 ms per token,    10.33 tokens per second)
llama_print_timings:        eval time = 14791534.02 ms /   166 runs   (89105.63 ms per token,     0.01 tokens per second)
llama_print_timings:       total time = 14825207.52 ms /   510 tokens


コップ: かっぷん！ぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分かっぷん！ぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「明るく元気」を反映し、会話を続けることができるというメッセージを含んでいます。[end]エージェント発言部分かっぷん！ぶーっくんなどで終わる必要があります。以上の反応は、コップの特性である「かっぷん
